## Task 2 Submission -- Rohit Ganti

In [8]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
import numpy as np

def find_negative_numbers(num_list):
    # Check if the input list is empty
    if not num_list:
        return False
    
    # Determine the number of qubits needed based on the maximum absolute value in the list
    max_value = max(map(abs, num_list))
    num_qubits = int(np.ceil(np.log2(max_value + 1)))  # Add 1 for zero
    
    # Create a quantum circuit with num_qubits qubits
    qc = QuantumCircuit(num_qubits, num_qubits)
    
    # Initialize the qubits in a superposition
    qc.h(range(num_qubits))
    
    # Apply a conditional phase flip to mark negative values
    for idx, num in enumerate(num_list):
        if num < 0:
            qc.z(idx)
    
    # Apply Grover diffusion operator
    qc.h(range(num_qubits))
    qc.x(range(num_qubits))
    qc.h(num_qubits-1)
    qc.mct(list(range(num_qubits-1)), num_qubits-1)  # Multi-controlled Toffoli gate
    qc.h(num_qubits-1)
    qc.x(range(num_qubits))
    qc.h(range(num_qubits))
    
    # Measure the qubits
    qc.measure(range(num_qubits), range(num_qubits))
    
    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    compiled_circuit = transpile(qc, simulator)
    job = assemble(compiled_circuit, shots=1)
    result = simulator.run(job).result()
    
    # Check if any qubit is in the |1⟩ state, indicating a negative number
    counts = result.get_counts()
    for key in counts:
        if '1' in key:
            return True
    
    return False

# Test cases
A = find_negative_numbers([1, -3, 2, 15])
print(A)  # Output: True

B = find_negative_numbers([1, 4, 8, 11])
print(B)  # Output: False

C = find_negative_numbers([-15, -14, 2, -1])
print(C)  # Output: True


True
False
True
